In [ ]:
import google.colab.drive as drive

In [ ]:
from transformers import XLNetTokenizer, XLNetForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn.functional as F
import numpy as np
import string
import copy
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW
from transformers import BertTokenizer, BertForSequenceClassification

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/Hands on Applied Analytics/train.csv')

In [ ]:
train_df.drop('Unnamed: 0', axis = 1, inplace = True)

In [ ]:
test_df = pd.read_csv('/content/drive/MyDrive/Hands on Applied Analytics/test.csv')

In [ ]:
test_df.drop('Unnamed: 0', axis = 1, inplace = True)

In [ ]:
import copy

In [ ]:
data = copy.deepcopy(train_df)

In [ ]:
data['Text'] = copy.deepcopy(data['cleaned_text'])
data['Label'] = copy.deepcopy(data['label'])

In [ ]:
from transformers import XLNetTokenizer, XLNetForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn.functional as F
import numpy as np
import string
import copy
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW



# Load pre-trained XLNet tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Assume data is your DataFrame with 'Text' and 'Label' columns
texts = data['Text'].tolist()  # Your list of text samples
labels = data['Label'].tolist()  # Corresponding labels for each text sample

# Encode labels
le = LabelEncoder()
labels_encoded = le.fit_transform(labels)
num_labels = len(le.classes_)
print(num_labels)
# Split the data into train and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels_encoded, test_size=0.2, random_state=85)

# Tokenize and encode the text data
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

# Convert the tokenized data to PyTorch tensors
train_dataset = torch.utils.data.TensorDataset(
    torch.tensor(train_encodings['input_ids']),
    torch.tensor(train_encodings['attention_mask']),
    torch.tensor(train_labels)
)
val_dataset = torch.utils.data.TensorDataset(
    torch.tensor(val_encodings['input_ids']),
    torch.tensor(val_encodings['attention_mask']),
    torch.tensor(val_labels)
)

# Load pre-trained XLNet model for sequence classification
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=num_labels)

# Define optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=1e-5)

# Define data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=8)

# Training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
model.to(device)

best_model_state_dict = None
best_val_accuracy = 0  # Track best validation accuracy
best_val_loss = np.inf  # Track best validation loss
early_stopping_patience = 3  # Number of epochs to wait for early stopping
early_stopping_counter = 0
training_losses = []
validation_losses = []
training_accuracies = []
validation_accuracies = []

for epoch in range(10):  # Change the number of epochs as needed
    model.train()
    train_losses = []
    train_preds = []
    train_true = []

    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        train_losses.append(loss.item())

        loss.backward()
        optimizer.step()

        preds = torch.argmax(outputs.logits, axis=1)
        train_preds.extend(preds.cpu().numpy())
        train_true.extend(labels.cpu().numpy())

    train_accuracy = accuracy_score(train_true, train_preds)
    avg_train_loss = np.mean(train_losses)

    # Validation
    model.eval()
    val_preds = []
    val_true = []
    val_losses = []

    for batch in val_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)

        logits = outputs.logits
        loss = F.cross_entropy(logits, labels)
        val_losses.append(loss.item())

        preds = torch.argmax(logits, axis=1)
        val_preds.extend(preds.cpu().numpy())
        val_true.extend(labels.cpu().numpy())

    val_accuracy = accuracy_score(val_true, val_preds)
    avg_val_loss = np.mean(val_losses)

    print(f'Epoch {epoch + 1}: Training Accuracy: {train_accuracy:.4f}, Training Loss: {avg_train_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}, Validation Loss: {avg_val_loss:.4f}')
    training_losses.append(avg_train_loss)
    validation_losses.append(avg_val_loss)
    training_accuracies.append(train_accuracy)
    validation_accuracies.append(val_accuracy)
    # Update best model
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        early_stopping_counter = 0
    else:
        early_stopping_counter += 1

    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        best_model_state_dict = copy.deepcopy(model.state_dict())
        print('New Best Model')

    if early_stopping_counter >= early_stopping_patience:
        print('Early stopping triggered.')
        break


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

4


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1: Training Accuracy: 0.9136, Training Loss: 0.2604, Validation Accuracy: 0.9279, Validation Loss: 0.2058
New Best Model
Epoch 2: Training Accuracy: 0.9440, Training Loss: 0.1648, Validation Accuracy: 0.9373, Validation Loss: 0.1864
New Best Model
Epoch 3: Training Accuracy: 0.9616, Training Loss: 0.1137, Validation Accuracy: 0.9364, Validation Loss: 0.1977
Epoch 4: Training Accuracy: 0.9745, Training Loss: 0.0761, Validation Accuracy: 0.9380, Validation Loss: 0.2143
New Best Model
Epoch 5: Training Accuracy: 0.9830, Training Loss: 0.0514, Validation Accuracy: 0.9342, Validation Loss: 0.2633
Early stopping triggered.


In [ ]:
model.load_state_dict(best_model_state_dict)
model.eval()
model.to(device)
# Evaluate on validation set
val_preds = []
val_true = []
for batch in val_loader:
    input_ids, attention_mask, labels = batch
    input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    preds = torch.argmax(logits, axis=1)
    val_preds.extend(preds.cpu().numpy())
    val_true.extend(labels.cpu().numpy())
val_preds = le.inverse_transform(val_preds)
val_true = le.inverse_transform(val_true)
# Print classification report
print(classification_report(val_true, val_preds))

              precision    recall  f1-score   support

           0       0.96      0.93      0.95      6038
           1       0.98      0.99      0.99      6015
           2       0.90      0.92      0.91      6076
           3       0.91      0.91      0.91      5871

    accuracy                           0.94     24000
   macro avg       0.94      0.94      0.94     24000
weighted avg       0.94      0.94      0.94     24000



In [ ]:
test_df

,text,label,cleaned_text
0,Fears for T N pension after talks Unions repre...,2,fears n pension talks unions representing work...
1,The Race is On: Second Private Team Sets Launc...,3,race second private team sets launch date huma...
2,Ky. Company Wins Grant to Study Peptides (AP) ...,3,ky company wins grant study peptides ap ap com...
3,Prediction Unit Helps Forecast Wildfires (AP) ...,3,prediction unit helps forecast wildfires ap ap...
4,Calif. Aims to Limit Farm-Related Smog (AP) AP...,3,calif aims limit farmrelated smog ap ap southe...
...,...,...,...
7595,Around the world Ukrainian presidential candid...,0,around world ukrainian presidential candidate ...
7596,Void is filled with Clement With the supply of...,1,void filled clement supply attractive pitching...
7597,Martinez leaves bitter Like Roger Clemens did ...,1,martinez leaves bitter like roger clemens almo...
7598,5 of arthritis patients in Singapore take Bext...,2,5 arthritis patients singapore take bextra cel...


In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
test_texts = test_df['cleaned_text'].tolist()  # Your list of text samples
test_labels = test_df['label'].tolist()  # Corresponding labels for each text sample

# test_labels_encoded = le.transform(test_labels)
le = LabelEncoder()
test_labels_encoded = le.fit_transform(labels)
num_labels = len(le.classes_)

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=num_labels)
model.load_state_dict(torch.load('/content/drive/MyDrive/Hands on Applied Analytics/distil_model_apr10.pt'))
model.eval()
model.to(device)
# Tokenize and encode the test text data
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

# Create test dataset
test_dataset = torch.utils.data.TensorDataset(
    torch.tensor(test_encodings['input_ids']),
    torch.tensor(test_encodings['attention_mask']),
    torch.tensor(test_labels_encoded)
)

# Define test data loader
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8)

# Evaluate on test set
test_preds = []
test_true = []
for batch in test_loader:
    input_ids, attention_mask, labels = batch
    input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    preds = torch.argmax(logits, axis=1)
    test_preds.extend(preds.cpu().numpy())
    test_true.extend(labels.cpu().numpy())

# Inverse transform the predicted and true labels
test_preds = le.inverse_transform(test_preds)
test_true = le.inverse_transform(test_true)

# Print classification report for test set
print(classification_report(test_true, test_preds))

              precision    recall  f1-score   support

           0       0.96      0.92      0.94      1900
           1       0.98      0.98      0.98      1900
           2       0.89      0.92      0.90      1900
           3       0.92      0.90      0.91      1900

    accuracy                           0.93      7600
   macro avg       0.93      0.93      0.93      7600
weighted avg       0.93      0.93      0.93      7600



In [ ]:
print(classification_report(test_true, test_preds))

              precision    recall  f1-score   support

           0       0.95      0.94      0.94      6038
           1       0.98      0.99      0.98      6015
           2       0.92      0.90      0.91      6076
           3       0.90      0.93      0.92      5871

    accuracy                           0.94     24000
   macro avg       0.94      0.94      0.94     24000
weighted avg       0.94      0.94      0.94     24000



In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/Hands on Applied Analytics/distil_model_apr10.pt')

In [ ]:
from transformers import XLNetTokenizer, XLNetForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn.functional as F
import numpy as np
import string
import copy
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW



# Load pre-trained XLNet tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Assume data is your DataFrame with 'Text' and 'Label' columns
texts = data['Text'].tolist()  # Your list of text samples
labels = data['Label'].tolist()  # Corresponding labels for each text sample

# Encode labels
le = LabelEncoder()
labels_encoded = le.fit_transform(labels)
num_labels = len(le.classes_)
# Split the data into train and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels_encoded, test_size=0.2, random_state=85)

# Tokenize and encode the text data
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

# Convert the tokenized data to PyTorch tensors
train_dataset = torch.utils.data.TensorDataset(
    torch.tensor(train_encodings['input_ids']),
    torch.tensor(train_encodings['attention_mask']),
    torch.tensor(train_labels)
)
val_dataset = torch.utils.data.TensorDataset(
    torch.tensor(val_encodings['input_ids']),
    torch.tensor(val_encodings['attention_mask']),
    torch.tensor(val_labels)
)

# Load pre-trained XLNet model for sequence classification
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=num_labels)

# Define optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=1e-5)

# Define data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=8)

# Training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
model.to(device)

best_model_state_dict = None
best_val_accuracy = 0  # Track best validation accuracy
best_val_loss = np.inf  # Track best validation loss
early_stopping_patience = 3  # Number of epochs to wait for early stopping
early_stopping_counter = 0
training_losses = []
validation_losses = []
training_accuracies = []
validation_accuracies = []

for epoch in range(10):  # Change the number of epochs as needed
    model.train()
    train_losses = []
    train_preds = []
    train_true = []

    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        train_losses.append(loss.item())

        loss.backward()
        optimizer.step()

        preds = torch.argmax(outputs.logits, axis=1)
        train_preds.extend(preds.cpu().numpy())
        train_true.extend(labels.cpu().numpy())

    train_accuracy = accuracy_score(train_true, train_preds)
    avg_train_loss = np.mean(train_losses)

    # Validation
    model.eval()
    val_preds = []
    val_true = []
    val_losses = []

    for batch in val_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)

        logits = outputs.logits
        loss = F.cross_entropy(logits, labels)
        val_losses.append(loss.item())

        preds = torch.argmax(logits, axis=1)
        val_preds.extend(preds.cpu().numpy())
        val_true.extend(labels.cpu().numpy())

    val_accuracy = accuracy_score(val_true, val_preds)
    avg_val_loss = np.mean(val_losses)

    print(f'Epoch {epoch + 1}: Training Accuracy: {train_accuracy:.4f}, Training Loss: {avg_train_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}, Validation Loss: {avg_val_loss:.4f}')
    training_losses.append(avg_train_loss)
    validation_losses.append(avg_val_loss)
    training_accuracies.append(train_accuracy)
    validation_accuracies.append(val_accuracy)
    # Update best model
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        early_stopping_counter = 0
    else:
        early_stopping_counter += 1

    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        best_model_state_dict = copy.deepcopy(model.state_dict())
        print('New Best Model')

    if early_stopping_counter >= early_stopping_patience:
        print('Early stopping triggered.')
        break


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1: Training Accuracy: 0.9054, Training Loss: 0.2792, Validation Accuracy: 0.9257, Validation Loss: 0.2235
New Best Model
Epoch 2: Training Accuracy: 0.9339, Training Loss: 0.1938, Validation Accuracy: 0.9357, Validation Loss: 0.1873
New Best Model
Epoch 3: Training Accuracy: 0.9484, Training Loss: 0.1496, Validation Accuracy: 0.9382, Validation Loss: 0.1891
New Best Model
Epoch 4: Training Accuracy: 0.9588, Training Loss: 0.1193, Validation Accuracy: 0.9389, Validation Loss: 0.1871
New Best Model
Epoch 5: Training Accuracy: 0.9667, Training Loss: 0.0955, Validation Accuracy: 0.9370, Validation Loss: 0.2034
Epoch 6: Training Accuracy: 0.9724, Training Loss: 0.0788, Validation Accuracy: 0.9335, Validation Loss: 0.2318
Epoch 7: Training Accuracy: 0.9780, Training Loss: 0.0635, Validation Accuracy: 0.9306, Validation Loss: 0.2554
Early stopping triggered.


In [ ]:
model.load_state_dict(best_model_state_dict)
model.eval()
model.to(device)
# Evaluate on validation set
val_preds = []
val_true = []
for batch in val_loader:
    input_ids, attention_mask, labels = batch
    input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    preds = torch.argmax(logits, axis=1)
    val_preds.extend(preds.cpu().numpy())
    val_true.extend(labels.cpu().numpy())
val_preds = le.inverse_transform(val_preds)
val_true = le.inverse_transform(val_true)
# Print classification report
print(classification_report(val_true, val_preds))

              precision    recall  f1-score   support

           0       0.95      0.94      0.95      6038
           1       0.98      0.99      0.98      6015
           2       0.92      0.90      0.91      6076
           3       0.90      0.93      0.91      5871

    accuracy                           0.94     24000
   macro avg       0.94      0.94      0.94     24000
weighted avg       0.94      0.94      0.94     24000



In [ ]:
test_texts = test_df['cleaned_text'].tolist()  # Your list of text samples
test_labels = test_df['label'].tolist()  # Corresponding labels for each text sample

test_labels_encoded = le.transform(test_labels)

# Tokenize and encode the test text data
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

# Create test dataset
test_dataset = torch.utils.data.TensorDataset(
    torch.tensor(test_encodings['input_ids']),
    torch.tensor(test_encodings['attention_mask']),
    torch.tensor(test_labels_encoded)
)

# Define test data loader
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8)

# Evaluate on test set
test_preds = []
test_true = []
for batch in test_loader:
    input_ids, attention_mask, labels = batch
    input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    preds = torch.argmax(logits, axis=1)
    test_preds.extend(preds.cpu().numpy())
    test_true.extend(labels.cpu().numpy())

# Inverse transform the predicted and true labels
test_preds = le.inverse_transform(test_preds)
test_true = le.inverse_transform(test_true)

# Print classification report for test set
print(classification_report(test_true, test_preds))

              precision    recall  f1-score   support

           0       0.95      0.94      0.94      1900
           1       0.97      0.98      0.98      1900
           2       0.92      0.89      0.90      1900
           3       0.90      0.93      0.91      1900

    accuracy                           0.93      7600
   macro avg       0.93      0.93      0.93      7600
weighted avg       0.93      0.93      0.93      7600



In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/Hands on Applied Analytics/roberta_model_apr11.pt')

In [ ]:
from transformers import XLNetTokenizer, XLNetForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn.functional as F
import numpy as np
import string
import copy
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW



# Load pre-trained XLNet tokenizer
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')

# Assume data is your DataFrame with 'Text' and 'Label' columns
texts = data['Text'].tolist()  # Your list of text samples
labels = data['Label'].tolist()  # Corresponding labels for each text sample

# Encode labels
le = LabelEncoder()
labels_encoded = le.fit_transform(labels)
num_labels = len(le.classes_)
# Split the data into train and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels_encoded, test_size=0.2, random_state=85)

# Tokenize and encode the text data
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

# Convert the tokenized data to PyTorch tensors
train_dataset = torch.utils.data.TensorDataset(
    torch.tensor(train_encodings['input_ids']),
    torch.tensor(train_encodings['attention_mask']),
    torch.tensor(train_labels)
)
val_dataset = torch.utils.data.TensorDataset(
    torch.tensor(val_encodings['input_ids']),
    torch.tensor(val_encodings['attention_mask']),
    torch.tensor(val_labels)
)

# Load pre-trained XLNet model for sequence classification
model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels=num_labels)

# Define optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=1e-5)

# Define data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=8)

# Training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
model.to(device)

best_model_state_dict = None
best_val_accuracy = 0  # Track best validation accuracy
best_val_loss = np.inf  # Track best validation loss
early_stopping_patience = 3  # Number of epochs to wait for early stopping
early_stopping_counter = 0
training_losses = []
validation_losses = []
training_accuracies = []
validation_accuracies = []

for epoch in range(10):  # Change the number of epochs as needed
    model.train()
    train_losses = []
    train_preds = []
    train_true = []

    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        train_losses.append(loss.item())

        loss.backward()
        optimizer.step()

        preds = torch.argmax(outputs.logits, axis=1)
        train_preds.extend(preds.cpu().numpy())
        train_true.extend(labels.cpu().numpy())

    train_accuracy = accuracy_score(train_true, train_preds)
    avg_train_loss = np.mean(train_losses)

    # Validation
    model.eval()
    val_preds = []
    val_true = []
    val_losses = []

    for batch in val_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)

        logits = outputs.logits
        loss = F.cross_entropy(logits, labels)
        val_losses.append(loss.item())

        preds = torch.argmax(logits, axis=1)
        val_preds.extend(preds.cpu().numpy())
        val_true.extend(labels.cpu().numpy())

    val_accuracy = accuracy_score(val_true, val_preds)
    avg_val_loss = np.mean(val_losses)

    print(f'Epoch {epoch + 1}: Training Accuracy: {train_accuracy:.4f}, Training Loss: {avg_train_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}, Validation Loss: {avg_val_loss:.4f}')
    training_losses.append(avg_train_loss)
    validation_losses.append(avg_val_loss)
    training_accuracies.append(train_accuracy)
    validation_accuracies.append(val_accuracy)
    # Update best model
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        early_stopping_counter = 0
    else:
        early_stopping_counter += 1

    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        best_model_state_dict = copy.deepcopy(model.state_dict())
        print('New Best Model')

    if early_stopping_counter >= early_stopping_patience:
        print('Early stopping triggered.')
        break


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


pytorch_model.bin:   0%|          | 0.00/467M [00:00<?, ?B/s]

Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1: Training Accuracy: 0.9025, Training Loss: 0.2904, Validation Accuracy: 0.9202, Validation Loss: 0.2281
New Best Model
Epoch 2: Training Accuracy: 0.9330, Training Loss: 0.1977, Validation Accuracy: 0.9317, Validation Loss: 0.2005
New Best Model
Epoch 3: Training Accuracy: 0.9497, Training Loss: 0.1493, Validation Accuracy: 0.9336, Validation Loss: 0.1991
New Best Model
Epoch 4: Training Accuracy: 0.9606, Training Loss: 0.1138, Validation Accuracy: 0.9333, Validation Loss: 0.2202
Epoch 5: Training Accuracy: 0.9699, Training Loss: 0.0879, Validation Accuracy: 0.9293, Validation Loss: 0.2624
Epoch 6: Training Accuracy: 0.9763, Training Loss: 0.0682, Validation Accuracy: 0.9243, Validation Loss: 0.2609
Early stopping triggered.


In [ ]:
model.load_state_dict(best_model_state_dict)
model.eval()
model.to(device)
# Evaluate on validation set
val_preds = []
val_true = []
for batch in val_loader:
    input_ids, attention_mask, labels = batch
    input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    preds = torch.argmax(logits, axis=1)
    val_preds.extend(preds.cpu().numpy())
    val_true.extend(labels.cpu().numpy())
val_preds = le.inverse_transform(val_preds)
val_true = le.inverse_transform(val_true)
# Print classification report
print(classification_report(val_true, val_preds))

              precision    recall  f1-score   support

           0       0.95      0.93      0.94      6038
           1       0.99      0.97      0.98      6015
           2       0.92      0.90      0.91      6076
           3       0.88      0.94      0.91      5871

    accuracy                           0.93     24000
   macro avg       0.93      0.93      0.93     24000
weighted avg       0.93      0.93      0.93     24000



In [ ]:
test_texts = test_df['cleaned_text'].tolist()  # Your list of text samples
test_labels = test_df['label'].tolist()  # Corresponding labels for each text sample

test_labels_encoded = le.transform(test_labels)

# Tokenize and encode the test text data
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

# Create test dataset
test_dataset = torch.utils.data.TensorDataset(
    torch.tensor(test_encodings['input_ids']),
    torch.tensor(test_encodings['attention_mask']),
    torch.tensor(test_labels_encoded)
)

# Define test data loader
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8)

# Evaluate on test set
test_preds = []
test_true = []
for batch in test_loader:
    input_ids, attention_mask, labels = batch
    input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    preds = torch.argmax(logits, axis=1)
    test_preds.extend(preds.cpu().numpy())
    test_true.extend(labels.cpu().numpy())

# Inverse transform the predicted and true labels
test_preds = le.inverse_transform(test_preds)
test_true = le.inverse_transform(test_true)

# Print classification report for test set
print(classification_report(test_true, test_preds))

              precision    recall  f1-score   support

           0       0.95      0.92      0.93      1900
           1       0.98      0.97      0.98      1900
           2       0.91      0.90      0.90      1900
           3       0.89      0.93      0.91      1900

    accuracy                           0.93      7600
   macro avg       0.93      0.93      0.93      7600
weighted avg       0.93      0.93      0.93      7600



In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/Hands on Applied Analytics/xlnet_model_apr13.pt')

In [ ]:
from transformers import XLNetTokenizer, XLNetForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn.functional as F
import numpy as np
import string
import copy
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW
from transformers import BertTokenizer, BertForSequenceClassification


# Load pre-trained XLNet tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Assume data is your DataFrame with 'Text' and 'Label' columns
texts = data['Text'].tolist()  # Your list of text samples
labels = data['Label'].tolist()  # Corresponding labels for each text sample

# Encode labels
le = LabelEncoder()
labels_encoded = le.fit_transform(labels)
num_labels = len(le.classes_)
# Split the data into train and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels_encoded, test_size=0.2, random_state=85)

# Tokenize and encode the text data
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

# Convert the tokenized data to PyTorch tensors
train_dataset = torch.utils.data.TensorDataset(
    torch.tensor(train_encodings['input_ids']),
    torch.tensor(train_encodings['attention_mask']),
    torch.tensor(train_labels)
)
val_dataset = torch.utils.data.TensorDataset(
    torch.tensor(val_encodings['input_ids']),
    torch.tensor(val_encodings['attention_mask']),
    torch.tensor(val_labels)
)

# Load pre-trained XLNet model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)

# Define optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=1e-5)

# Define data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=8)

# Training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
model.to(device)

best_model_state_dict = None
best_val_accuracy = 0  # Track best validation accuracy
best_val_loss = np.inf  # Track best validation loss
early_stopping_patience = 3  # Number of epochs to wait for early stopping
early_stopping_counter = 0
training_losses = []
validation_losses = []
training_accuracies = []
validation_accuracies = []

for epoch in range(10):  # Change the number of epochs as needed
    model.train()
    train_losses = []
    train_preds = []
    train_true = []

    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        train_losses.append(loss.item())

        loss.backward()
        optimizer.step()

        preds = torch.argmax(outputs.logits, axis=1)
        train_preds.extend(preds.cpu().numpy())
        train_true.extend(labels.cpu().numpy())

    train_accuracy = accuracy_score(train_true, train_preds)
    avg_train_loss = np.mean(train_losses)

    # Validation
    model.eval()
    val_preds = []
    val_true = []
    val_losses = []

    for batch in val_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)

        logits = outputs.logits
        loss = F.cross_entropy(logits, labels)
        val_losses.append(loss.item())

        preds = torch.argmax(logits, axis=1)
        val_preds.extend(preds.cpu().numpy())
        val_true.extend(labels.cpu().numpy())

    val_accuracy = accuracy_score(val_true, val_preds)
    avg_val_loss = np.mean(val_losses)

    print(f'Epoch {epoch + 1}: Training Accuracy: {train_accuracy:.4f}, Training Loss: {avg_train_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}, Validation Loss: {avg_val_loss:.4f}')
    training_losses.append(avg_train_loss)
    validation_losses.append(avg_val_loss)
    training_accuracies.append(train_accuracy)
    validation_accuracies.append(val_accuracy)
    # Update best model
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        early_stopping_counter = 0
    else:
        early_stopping_counter += 1

    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        best_model_state_dict = copy.deepcopy(model.state_dict())
        print('New Best Model')

    if early_stopping_counter >= early_stopping_patience:
        print('Early stopping triggered.')
        break


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1: Training Accuracy: 0.9145, Training Loss: 0.2523, Validation Accuracy: 0.9278, Validation Loss: 0.2037
New Best Model
Epoch 2: Training Accuracy: 0.9457, Training Loss: 0.1573, Validation Accuracy: 0.9381, Validation Loss: 0.1763
New Best Model
Epoch 3: Training Accuracy: 0.9637, Training Loss: 0.1062, Validation Accuracy: 0.9413, Validation Loss: 0.1882
New Best Model
Epoch 4: Training Accuracy: 0.9747, Training Loss: 0.0727, Validation Accuracy: 0.9390, Validation Loss: 0.2097
Epoch 5: Training Accuracy: 0.9836, Training Loss: 0.0491, Validation Accuracy: 0.9382, Validation Loss: 0.2256
Early stopping triggered.


In [ ]:
model.load_state_dict(best_model_state_dict)
model.eval()
model.to(device)
# Evaluate on validation set
val_preds = []
val_true = []
for batch in val_loader:
    input_ids, attention_mask, labels = batch
    input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    preds = torch.argmax(logits, axis=1)
    val_preds.extend(preds.cpu().numpy())
    val_true.extend(labels.cpu().numpy())
val_preds = le.inverse_transform(val_preds)
val_true = le.inverse_transform(val_true)
# Print classification report
print(classification_report(val_true, val_preds))

              precision    recall  f1-score   support

           0       0.96      0.93      0.95      6038
           1       0.98      0.99      0.99      6015
           2       0.93      0.90      0.91      6076
           3       0.90      0.94      0.92      5871

    accuracy                           0.94     24000
   macro avg       0.94      0.94      0.94     24000
weighted avg       0.94      0.94      0.94     24000



In [ ]:
test_texts = test_df['cleaned_text'].tolist()  # Your list of text samples
test_labels = test_df['label'].tolist()  # Corresponding labels for each text sample

test_labels_encoded = le.transform(test_labels)

# Tokenize and encode the test text data
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

# Create test dataset
test_dataset = torch.utils.data.TensorDataset(
    torch.tensor(test_encodings['input_ids']),
    torch.tensor(test_encodings['attention_mask']),
    torch.tensor(test_labels_encoded)
)

# Define test data loader
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8)

# Evaluate on test set
test_preds = []
test_true = []
for batch in test_loader:
    input_ids, attention_mask, labels = batch
    input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    preds = torch.argmax(logits, axis=1)
    test_preds.extend(preds.cpu().numpy())
    test_true.extend(labels.cpu().numpy())

# Inverse transform the predicted and true labels
test_preds = le.inverse_transform(test_preds)
test_true = le.inverse_transform(test_true)

# Print classification report for test set
print(classification_report(test_true, test_preds))

              precision    recall  f1-score   support

           0       0.95      0.94      0.95      1900
           1       0.98      0.98      0.98      1900
           2       0.93      0.90      0.91      1900
           3       0.90      0.93      0.92      1900

    accuracy                           0.94      7600
   macro avg       0.94      0.94      0.94      7600
weighted avg       0.94      0.94      0.94      7600



In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/Hands on Applied Analytics/ber_model_apr14.pt')

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
test_texts = test_df['cleaned_text'].tolist()  # Your list of text samples
test_labels = test_df['label'].tolist()  # Corresponding labels for each text sample

# test_labels_encoded = le.transform(test_labels)
le = LabelEncoder()
test_labels_encoded = le.fit_transform(test_labels)
num_labels = len(le.classes_)

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=num_labels)
model.load_state_dict(torch.load('/content/drive/MyDrive/Hands on Applied Analytics/distil_model_apr10.pt'))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.eval()
model.to(device)
# Tokenize and encode the test text data
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

# Create test dataset
test_dataset = torch.utils.data.TensorDataset(
    torch.tensor(test_encodings['input_ids']),
    torch.tensor(test_encodings['attention_mask']),
    torch.tensor(test_labels_encoded)
)

# Define test data loader
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8)

# Evaluate on test set
test_preds = []
test_true = []
for batch in test_loader:
    input_ids, attention_mask, labels = batch
    input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    preds = torch.argmax(logits, axis=1)
    test_preds.extend(preds.cpu().numpy())
    test_true.extend(labels.cpu().numpy())
    if preds.cpu().numpy() != labels.cpu().numpy():
        print(preds.cpu().numpy())
        print(labels.cpu().numpy())
        print(test_texts[0])
    else:
        print('incorrect')

# Inverse transform the predicted and true labels
test_preds = le.inverse_transform(test_preds)
test_true = le.inverse_transform(test_true)

# Print classification report for test set
print(classification_report(test_true, test_preds))

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


              precision    recall  f1-score   support

           0       0.95      0.94      0.94      1900
           1       0.98      0.98      0.98      1900
           2       0.92      0.89      0.90      1900
           3       0.90      0.93      0.91      1900

    accuracy                           0.94      7600
   macro avg       0.94      0.94      0.94      7600
weighted avg       0.94      0.94      0.94      7600



In [ ]:
accuracy_score(test_true, test_preds)

0.9355263157894737

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
test_texts = test_df['cleaned_text'].tolist()  # Your list of text samples
test_labels = test_df['label'].tolist()  # Corresponding labels for each text sample

# test_labels_encoded = le.transform(test_labels)
le = LabelEncoder()
test_labels_encoded = le.fit_transform(test_labels)
num_labels = len(le.classes_)

model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=num_labels)
model.load_state_dict(torch.load('/content/drive/MyDrive/Hands on Applied Analytics/roberta_model_apr11.pt'))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.eval()
model.to(device)
# Tokenize and encode the test text data
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

# Create test dataset
test_dataset = torch.utils.data.TensorDataset(
    torch.tensor(test_encodings['input_ids']),
    torch.tensor(test_encodings['attention_mask']),
    torch.tensor(test_labels_encoded)
)

# Define test data loader
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8)

# Evaluate on test set
test_preds = []
test_true = []
for batch in test_loader:
    input_ids, attention_mask, labels = batch
    input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    preds = torch.argmax(logits, axis=1)
    test_preds.extend(preds.cpu().numpy())
    test_true.extend(labels.cpu().numpy())

# Inverse transform the predicted and true labels
test_preds = le.inverse_transform(test_preds)
test_true = le.inverse_transform(test_true)

# Print classification report for test set
print(classification_report(test_true, test_preds))

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


              precision    recall  f1-score   support

           0       0.92      0.95      0.93      1900
           1       0.97      0.98      0.98      1900
           2       0.91      0.90      0.90      1900
           3       0.92      0.89      0.91      1900

    accuracy                           0.93      7600
   macro avg       0.93      0.93      0.93      7600
weighted avg       0.93      0.93      0.93      7600



In [ ]:
accuracy_score(test_true, test_preds)

0.930921052631579

In [ ]:
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
test_texts = test_df['cleaned_text'].tolist()  # Your list of text samples
test_labels = test_df['label'].tolist()  # Corresponding labels for each text sample

# test_labels_encoded = le.transform(test_labels)
le = LabelEncoder()
test_labels_encoded = le.fit_transform(test_labels)
num_labels = len(le.classes_)

model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels=num_labels)
model.load_state_dict(torch.load('/content/drive/MyDrive/Hands on Applied Analytics/xlnet_model_apr13.pt'))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.eval()
model.to(device)
# Tokenize and encode the test text data
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

# Create test dataset
test_dataset = torch.utils.data.TensorDataset(
    torch.tensor(test_encodings['input_ids']),
    torch.tensor(test_encodings['attention_mask']),
    torch.tensor(test_labels_encoded)
)

# Define test data loader
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8)

# Evaluate on test set
test_preds = []
test_true = []
for batch in test_loader:
    input_ids, attention_mask, labels = batch
    input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    preds = torch.argmax(logits, axis=1)
    test_preds.extend(preds.cpu().numpy())
    test_true.extend(labels.cpu().numpy())

# Inverse transform the predicted and true labels
test_preds = le.inverse_transform(test_preds)
test_true = le.inverse_transform(test_true)

# Print classification report for test set
print(classification_report(test_true, test_preds))

pytorch_model.bin:   0%|          | 0.00/467M [00:00<?, ?B/s]

Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


              precision    recall  f1-score   support

           0       0.95      0.92      0.93      1900
           1       0.98      0.97      0.98      1900
           2       0.91      0.90      0.90      1900
           3       0.89      0.93      0.91      1900

    accuracy                           0.93      7600
   macro avg       0.93      0.93      0.93      7600
weighted avg       0.93      0.93      0.93      7600



In [ ]:
accuracy_score(test_true, test_preds)

0.9292105263157895

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
test_texts = test_df['cleaned_text'].tolist()  # Your list of text samples
test_labels = test_df['label'].tolist()  # Corresponding labels for each text sample

# test_labels_encoded = le.transform(test_labels)
le = LabelEncoder()
test_labels_encoded = le.fit_transform(test_labels)
num_labels = len(le.classes_)

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)
model.load_state_dict(torch.load('/content/drive/MyDrive/Hands on Applied Analytics/ber_model_apr14.pt'))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.eval()
model.to(device)
# Tokenize and encode the test text data
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

# Create test dataset
test_dataset = torch.utils.data.TensorDataset(
    torch.tensor(test_encodings['input_ids']),
    torch.tensor(test_encodings['attention_mask']),
    torch.tensor(test_labels_encoded)
)

# Define test data loader
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8)

# Evaluate on test set
test_preds = []
test_true = []
for batch in test_loader:
    input_ids, attention_mask, labels = batch
    input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    preds = torch.argmax(logits, axis=1)
    test_preds.extend(preds.cpu().numpy())
    test_true.extend(labels.cpu().numpy())

# Inverse transform the predicted and true labels
test_preds = le.inverse_transform(test_preds)
test_true = le.inverse_transform(test_true)

# Print classification report for test set
print(classification_report(test_true, test_preds))

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


              precision    recall  f1-score   support

           0       0.95      0.94      0.95      1900
           1       0.98      0.98      0.98      1900
           2       0.93      0.90      0.91      1900
           3       0.90      0.93      0.92      1900

    accuracy                           0.94      7600
   macro avg       0.94      0.94      0.94      7600
weighted avg       0.94      0.94      0.94      7600



In [ ]:
accuracy_score(test_true, test_preds)

0.9397368421052632